In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sparknlp
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=f09453dacdc50eab1a0838a5dbe91282539f50520c409b824206eec301daf285
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, CountVectorizer, NGram, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from pyspark.ml.feature import Word2Vec
from pyspark.sql import functions as F

from pyspark.ml.classification import MultilayerPerceptronClassifier

from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, Word2Vec, StringIndexer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# Khởi tạo SparkSession
spark = SparkSession.builder.appName("sentimentanalysis").getOrCreate()

# Đọc dữ liệu
df = spark.read.option("multiLine", True).option("header", True).option("escape", "\"").csv("twitter.csv")

# Hiển thị dữ liệu
df.show(10)

+-------------------+--------------+--------------------+--------------+
|          timestamp|          user|                text|user_sentiment|
+-------------------+--------------+--------------------+--------------+
|2021-07-25 16:30:26|    banditlink| Gigachad Carlson...|      positive|
|2021-07-25 16:30:30|   schmelonbun| wei ying with hi...|      positive|
|2021-07-25 16:30:31|      0luis200|              Fuck U|       neutral|
|2021-07-25 16:30:30|    TrimbleSam| if you push voti...|      negative|
|2021-07-25 16:30:25|   John_weber_| if you push voti...|      negative|
|2021-07-25 16:30:32|trunghieuveluv| Today is WanD! ?...|      positive|
|2021-07-25 16:30:30|  DavidKister1| Republicans who ...|      negative|
|2021-07-25 16:30:26|dragon_lady_51| Republicans who ...|      negative|
|2021-07-25 16:30:27|      powwhead| Republicans who ...|      negative|
|2021-07-25 16:30:21| Rajeh44961284|portable charging...|      positive|
+-------------------+--------------+---------------

In [ ]:
# Tạo train set và test set
data = df.randomSplit([0.8, 0.2])
train_set = data[0]
test_set = data[1]
train_count = train_set.count()
test_count = test_set.count()
print("Train Count:", train_count)
print("Test Count:", test_count)

Train Count: 8011
Test Count: 1987


In [ ]:
# Mô hình LSTM
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import RegexTokenizer, Word2Vec, StringIndexer


In [ ]:
# Tiền xử lý dữ liệu cho mô hình LSTM
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
wordsData = tokenizer.transform(train_set).select("words", "user_sentiment")

In [ ]:
# Tạo Word2Vec embedding
word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="features")
word2VecModel = word2Vec.fit(wordsData)
result = word2VecModel.transform(wordsData)


In [ ]:
labelIndexer = StringIndexer(inputCol="user_sentiment", outputCol="label", handleInvalid="skip") \
    .setStringOrderType("frequencyDesc")

In [ ]:

tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
wordsData = tokenizer.transform(train_set).select("words", "user_sentiment")
word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="features")
word2VecModel = word2Vec.fit(wordsData)
result = word2VecModel.transform(wordsData)
labelIndexer = StringIndexer(inputCol="user_sentiment", outputCol="label", handleInvalid="skip") \
    .setStringOrderType("frequencyDesc")
layers = [100, 64, 32, 3]
dropout_rates = [0.2, 0.2, 0.2]
input_dim = len(indexedData.select("features").first()[0])

layers_with_dropout = []
for layer, dropout_rate in zip(layers[:-1], dropout_rates):
    layers_with_dropout.append(layer)
    layers_with_dropout.append(layer - int(layer * dropout_rate))
layers_with_dropout.append(layers[-1])

# Tạo mô hình MLP với lớp Dropout
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=layers_with_dropout, blockSize=1000, seed=1234,
                                     featuresCol="input_features", labelCol="label")




In [ ]:
# Tạo pipeline với VectorAssembler và mô hình MLP
pipeline = Pipeline(stages=[vectorAssembler, mlp])
# Xây dựng pipeline
pipeline = Pipeline(stages=[tokenizer, word2Vec, labelIndexer, trainer])

In [ ]:
# Huấn luyện mô hình LSTM
model_lstm = pipeline.fit(train_set)

In [ ]:
# Đánh giá mô hình trên tập kiểm tra
predictions_lstm = model_lstm.transform(test_set)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lstm = evaluator.evaluate(predictions_lstm)
print("Accuracy Score (LSTM): {:.4f}".format(accuracy_lstm))

Accuracy Score (LSTM): 0.7740


In [ ]:
# Đánh giá mô hình trên tập kiểm tra và hiển thị dữ liệu dự báo
predictions_lstm = model_lstm.transform(test_set)
predictions_lstm.show()


+-------------------+---------------+--------------------+--------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|          timestamp|           user|                text|user_sentiment|               words|            features|label|       rawPrediction|         probability|prediction|
+-------------------+---------------+--------------------+--------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|2021-07-25 16:30:23|  YourNewsCity1|Apple iPhone 13 S...|      positive|[apple, iphone, 1...|[0.03023655911674...|  0.0|[2.42848783420747...|[0.65341827285394...|       0.0|
|2021-07-25 16:30:25|BeadingfanNancy| It’s really quit...|      negative|[it, s, really, q...|[-0.0329044269165...|  1.0|[0.62550652531956...|[0.00584170057851...|       1.0|
|2021-07-25 16:30:34|   factormystic|ted lasso is an i...|      positive|[ted, lasso, is, ...|[-0.0051511555858...|  0.0|[2.1